# Kakaomap API 받기
### 도로명주소로부터 위도, 경도 구하기
https://apis.map.kakao.com/ 에서 APP KEY 발급 => 애플리케이션 추가하기<br>

In [1]:
import requests
from urllib.parse import quote
import pandas as pd

In [2]:
# key 가져오기
# https://developers.kakao.com/console/app/506999

# 발급받은 4개의 키 중 restAPI를 txt파일에 담으면 된다.
key_fd = open('../keys/kakaomapkey.txt', mode='r', encoding='utf8')
kmap_key = key_fd.read(100)
key_fd.close()

In [3]:
addr = '서울특별시 강서구 화곡로 302 (화곡동)'

# https://developers.kakao.com/docs/latest/ko/daum-search/dev-guide
# 에서 키 집어넣는 방법 알려줌
# http://blog.daum.net/geoscience/1436


url = 'https://dapi.kakao.com/v2/local/search/address.json?query='+quote(addr)

result = requests.get(url, headers={'Authorization':'KakaoAK '+kmap_key}).json() # KakaoAK 뒤 space하나 조심!

In [4]:
result

{'documents': [{'address': {'address_name': '서울 강서구 화곡동 980-16',
    'b_code': '1150010300',
    'h_code': '1150059100',
    'main_address_no': '980',
    'mountain_yn': 'N',
    'region_1depth_name': '서울',
    'region_2depth_name': '강서구',
    'region_3depth_h_name': '화곡6동',
    'region_3depth_name': '화곡동',
    'sub_address_no': '16',
    'x': '126.849532173376',
    'y': '37.5509655144007'},
   'address_name': '서울 강서구 화곡로 302',
   'address_type': 'ROAD_ADDR',
   'road_address': {'address_name': '서울 강서구 화곡로 302',
    'building_name': '강서구청',
    'main_building_no': '302',
    'region_1depth_name': '서울',
    'region_2depth_name': '강서구',
    'region_3depth_name': '화곡동',
    'road_name': '화곡로',
    'sub_building_no': '',
    'underground_yn': 'N',
    'x': '126.849532173376',
    'y': '37.5509655144007',
    'zone_no': '07658'},
   'x': '126.849532173376',
   'y': '37.5509655144007'}],
 'meta': {'is_end': True, 'pageable_count': 1, 'total_count': 1}}

In [5]:
# 경도 x, 위도 y
lat = result['documents'][0]['y']
lng = result['documents'][0]['x']
lat, lng

('37.5509655144007', '126.849532173376')

### address.csv 파일에 위도, 경도 추가하기

In [6]:
df = pd.read_csv('address.csv')
df

,구청,주소
0,강서구청,서울특별시 강서구 화곡로 302 (화곡동)
1,양천구청,서울특별시 양천구 목동동로 105 (신정동)
2,부천시청/부천시의회,경기도 부천시 길주로 210 (중동)
3,영등포구청,서울특별시 영등포구 당산로 123 (당산동3가)
4,김포시청,경기도 김포시 사우중로 1 (사우동)


In [7]:
# for문 돌릴 때는 df의 index를 활용하면 쉽다.
df.index

RangeIndex(start=0, stop=5, step=1)

In [8]:
lat_list = [] ; lng_list = []

for index in df.index:
    addr = df['주소'][index]
    url = 'https://dapi.kakao.com/v2/local/search/address.json?query='+addr

    result = requests.get(url, headers={'Authorization':'KakaoAK '+kmap_key}).json() 
    lat_list.append(result['documents'][0]['y'])
    lng_list.append(result['documents'][0]['x'])


NameError: name 'urlparse' is not defined

In [33]:
df['lat'] = lat_list
df['lng'] = lng_list

In [34]:
df

,구청,주소,lat,lng
0,강서구청,서울특별시 강서구 화곡로 302 (화곡동),37.5509655144007,126.849532173376
1,양천구청,서울특별시 양천구 목동동로 105 (신정동),37.5170753784215,126.866542541936
2,부천시청/부천시의회,경기도 부천시 길주로 210 (중동),37.5035810269294,126.765337160328
3,영등포구청,서울특별시 영등포구 당산로 123 (당산동3가),37.525963157053,126.896367130558
4,김포시청,경기도 김포시 사우중로 1 (사우동),37.6151899959384,126.715717006327


In [35]:
df.to_csv('address2.csv',sep=',',encoding='utf8', index=False)